[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/bamirandab/nlp-disaster-tweets/blob/main/notebooks/model.ipynb)

In [1]:
from google.colab import files
files.upload() #upload kaggle.json

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"bamirandab","key":"acf8e7fbd28901f7d9fc992f669fd7d0"}'}

In [2]:
from google.colab import files
files.upload() #upload clean.py

Saving clean.py to clean.py


{'clean.py': b'import re\r\nimport nltk\r\nnltk.download(\'punkt\')\r\nfrom nltk.corpus import stopwords\r\nfrom nltk.tokenize import word_tokenize\r\nimport mapply\r\n\r\n\r\nmapply.init(\r\n    n_workers=-1,\r\n    chunk_size=100,\r\n    max_chunks_per_worker=8,\r\n    progressbar=False\r\n)\r\n\r\nnltk.download(\'stopwords\')\r\nstop_words = stopwords.words()\r\n\r\ndef search_keywords(x, tweet):\r\n  result = re.search(\'[{letter}]\\w+\'.format(letter=x),tweet)\r\n  if result: \r\n    return True\r\n  else:\r\n    return False\r\n\r\ndef clean_tweet(tweet):\r\n  tweet_tokens = re.sub(\'\\s+\', \' \', tweet.lower()).split(\' \')\r\n  tweet_clean = \' \'.join([word for word in tweet_tokens if not word in stop_words])\r\n  return tweet_clean\r\n\r\n\r\ndef drop_word(x, word, tweet):\r\n  return re.sub(\'[{letter}]\\w+\'.format(letter=x),word,tweet)\r\n\r\ndef drop_url(tweet):\r\n  return re.sub(r\'http\\S+\',\'url\',tweet)\r\n\r\n\r\ndef clean_df(x):\r\n    x_clean_temp = x["text"].ma

In [3]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

kaggle.json


In [4]:
!kaggle competitions download -c nlp-getting-started -p ../data

  0% 0.00/593k [00:00<?, ?B/s]
100% 593k/593k [00:00<00:00, 99.3MB/s]


In [5]:
!pip install mapply

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76 kB 4.3 MB/s 
     |████████████████████████████████| 115 kB 40.3 MB/s 
     |████████████████████████████████| 52 kB 1.6 MB/s 


In [19]:
from clean import clean_df
from zipfile import ZipFile
import pandas as pd
import os
from sklearn.model_selection import train_test_split

In [7]:
path = os.path.join('../data/nlp-getting-started.zip')

with ZipFile(path, 'r') as zipObj:
#    # Extract all the contents of zip file in current directory
   zipObj.extractall('../data')

In [8]:
data_path = '../data/'
train_df = pd.read_csv(data_path + 'train.csv', encoding='utf-8')
test_df = pd.read_csv(data_path + 'test.csv', encoding='utf-8')

In [9]:
train_labels = train_df["target"].to_list()
train_sentences = clean_df(train_df)
test_sentences = clean_df(test_df)

In [10]:
# Number of examples to use for training
training_size = 20000

# Vocabulary size of the tokenizer
vocab_size = 10000

# Maximum length of the padded sequences
max_length = 32

# Output dimensions of the Embedding layer
embedding_dim = 16

In [12]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Parameters for padding and OOV tokens
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

# Initialize the Tokenizer class
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)

# Generate the word index dictionary
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index

# Generate and pad the training sequences
training_sequences = tokenizer.texts_to_sequences(train_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

# Generate and pad the testing sequences
testing_sequences = tokenizer.texts_to_sequences(test_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

# Convert the labels lists into numpy arrays
training_labels = np.array(train_labels)

In [27]:
X_train, X_test, y_train, y_test = train_test_split(training_padded,training_labels,stratify = training_labels, 
                                                    test_size=0.1, random_state=42)

In [39]:

# Build the model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
    # tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128,  return_sequences=True)),
    # tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    # tf.keras.layers.Dropout(0.5),
    # tf.keras.layers.Dense(128, activation='relu'),
    # tf.keras.layers.Dropout(0.5),
    # tf.keras.layers.Dense(1, activation='sigmoid')
])

# Print the model summary
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 32, 16)            160000    
                                                                 
 global_average_pooling1d_3   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_17 (Dense)            (None, 128)               2176      
                                                                 
 dropout_8 (Dropout)         (None, 128)               0         
                                                                 
 dense_18 (Dense)            (None, 64)                8256      
                                                                 
 dropout_9 (Dropout)         (None, 64)                0         
                                                      

In [40]:
# Compile the model
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [41]:
num_epochs = 30

# Train the model
history = model.fit(X_train, y_train, epochs=num_epochs, verbose=1,
                    batch_size=2**6 ,validation_data = (X_test, y_test))

Epoch 1/30
108/108 [==============================] - 1s 5ms/step - loss: 0.6791 - accuracy: 0.5685 - val_loss: 0.6659 - val_accuracy: 0.5709
Epoch 2/30
108/108 [==============================] - 0s 4ms/step - loss: 0.5756 - accuracy: 0.6980 - val_loss: 0.4941 - val_accuracy: 0.7717
Epoch 3/30
108/108 [==============================] - 0s 4ms/step - loss: 0.3770 - accuracy: 0.8476 - val_loss: 0.4678 - val_accuracy: 0.8005
Epoch 4/30
108/108 [==============================] - 0s 4ms/step - loss: 0.2884 - accuracy: 0.8885 - val_loss: 0.5013 - val_accuracy: 0.7979
Epoch 5/30
108/108 [==============================] - 0s 4ms/step - loss: 0.2426 - accuracy: 0.9095 - val_loss: 0.5402 - val_accuracy: 0.7861
Epoch 6/30
108/108 [==============================] - 0s 4ms/step - loss: 0.2003 - accuracy: 0.9254 - val_loss: 0.5978 - val_accuracy: 0.7756
Epoch 7/30
108/108 [==============================] - 0s 3ms/step - loss: 0.1690 - accuracy: 0.9428 - val_loss: 0.6495 - val_accuracy: 0.7638
Epoch 

In [42]:
testing_labels = model.predict(testing_padded)

In [55]:
test_labels = [1 if label > 0.40 else 0 for label in testing_labels ]

In [56]:
test_df['target'] = pd.Series(test_labels)
print(test_df.shape)
test_df[['id','target']].to_csv("test.csv", index=False)
test_df.target.sum()

(3263, 5)


1302